In [1]:
# -*- coding: utf-8 -*-
"""
Analyse des capacités de productions d'électricité en métropole  française

Created on Wed May 29 13:54:11 2024

@author: Thierry ALLEM
"""

"\nAnalyse des capacités de productions d'électricité en métropole  française\n\nCreated on Wed May 29 13:54:11 2024\n\n@author: Thierry ALLEM\n"

In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm # Importation de tqdm pour la barre de progression
import chardet

In [3]:
# Lecture des premiers 10000 octets pour détecter l'encodage
with open('registre_national_installation_production_stockage_electricite.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))
    print(result)  # Affiche l'encodage détecté

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


In [4]:
# Chargement du fichier du registre national des installations de production et stockage d'électricité
df_cap_prod = pd.read_csv('registre_national_installation_production_stockage_electricite.csv', sep=';',encoding= 'UTF-8-SIG', low_memory=False)
df_cap_prod.head()

,nomInstallation,codeEICResourceObject,codeIRIS,codeINSEECommune,commune,codeEPCI,EPCI,codeDepartement,departement,codeRegion,...,capaciteReservoir,hauteurChute,productible,debitMaximal,codeGestionnaire,gestionnaire,energieAnnuelleGlissanteInjectee,energieAnnuelleGlissanteProduite,energieAnnuelleGlissanteSoutiree,energieAnnuelleGlissanteStockee
0,Confidentiel,17W000000034442R,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN
1,Confidentiel,17W0000000383544,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN
2,Confidentiel,17W000000032988U,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN
3,Confidentiel,17W000001336047V,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN
4,Confidentiel,17W000001336508L,2A0010000,2A001,Afa,242010056,CA du Pays Ajaccien,2A,Corse-du-Sud,94.0,...,NaN,NaN,NaN,NaN,23X160203-000021,EDF-SEI,0.0,NaN,0.0,NaN


In [5]:
# Export des 100 premières lignes dans un nouveau fichier CSV pour aperçu
df_cap_prod_head = df_cap_prod.head(100)
df_cap_prod_head.to_csv('registre_national_installation_production_first_100_lines.csv', sep=';', encoding='latin-1', index=False)

In [6]:
#   Renommage des colonnes

#     Dictionnaire avec les nouveaux noms des colonnes

dictionnaire_entetes = {'nomInstallation ':'nom_installation',
                        'codeEICResourceObject':'code_EIC',
                        'codeRegion' : 'code_region',                
                'dateRaccordement' : 'date_raccordement',
                'dateDeraccordement' : 'date_deraccordement',
                'dateMiseEnService' : 'date_mise_en_service',
                'codeFiliere' :'code_filiere',
                'codeCombustible' : 'code_combustible',
                'codesCombustiblesSecondaires' : 'codes_combustibles_secondaires',
                'combustiblesSecondaires' : 'combustibles_secondaires',
                'codeTechnologie' : 'code_technologie',
                'typeStockage' : 'type_stockage',
                'puisMaxInstallee' : 'puis_max_installee',
                'puisMaxRacCharge' : 'puis_max_rac_charge',
                'puisMaxCharge' : 'puis_max_charge',
                'puisMaxRac' : 'puis_max_rac',
                'puisMaxInstalleeDisCharge' : 'puis_max_installee_discharge',
                'energieStockable' : 'energie_stockable',
                'energieAnnuelleGlissanteInjectee' : 'energie_annuelle_glissante_injectee',
                'energieAnnuelleGlissanteProduite' : 'energie_annuelle_glissante_produite',
                'energieAnnuelleGlissanteSoutiree' : 'energie_annuelle_glissante_soutiree',
                'energieAnnuelleGlissanteStockee' : 'energie_annuelle_glissante_stockee'}
                
df_cap_prod=df_cap_prod.rename(dictionnaire_entetes, axis = 1)
df_cap_prod['filiere'].unique()

array(['Solaire', 'Thermique non renouvelable', 'Hydraulique', 'Eolien',
       'Bioénergies', 'Stockage non hydraulique', 'Nucléaire',
       'thermique', 'stockage_batterie', 'Energies Marines', 'Géothermie',
       'eolien'], dtype=object)

In [7]:
# Affichage du nombre de lignes et colonnes du dataframe et décompte des types des valeurs

print("Nombre de lignes du dataframe : ", len(df_cap_prod.index))
print("Nombre de colonnes du dataframe : ", len(df_cap_prod.columns))
print("Valeurs uniques de la colonne 'region' :", df_cap_prod['region'].unique())
print("Types de valeurs :",df_cap_prod.dtypes.value_counts() )
df_cap_prod.info()


Nombre de lignes du dataframe :  73693
Nombre de colonnes du dataframe :  49
Valeurs uniques de la colonne 'region' : ['Corse' 'Auvergne-Rhône-Alpes' 'Hauts-de-France'
 "Provence-Alpes-Côte d'Azur" 'Grand Est' 'Occitanie' 'Normandie'
 'Nouvelle-Aquitaine' 'Centre-Val de Loire' 'Bourgogne-Franche-Comté'
 'Bretagne' 'Pays de la Loire' 'Île-de-France' 'Guadeloupe' 'Martinique'
 'Guyane' 'La Réunion' nan]
Types de valeurs : object     32
float64    16
int64       1
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73693 entries, 0 to 73692
Data columns (total 49 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   nomInstallation                      73693 non-null  object 
 1   code_EIC                             52279 non-null  object 
 2   codeIRIS                             43125 non-null  object 
 3   codeINSEECommune                     72397 non-null  object 
 4 

In [8]:
# DESCRIPTION DES COLONNES
#    Création d'un dataframe de description des colonnes de df_cap_prod, sans regroupement par region

df_colonnes = pd.DataFrame()

#   Stockage du noms des colonnes du  dataframe df_cap_prod
df_colonnes['Nom_colonne'] = list(df_cap_prod.columns)

#   Affichage du type des valeurs des colonnes de df
df_colonnes.index = df_cap_prod.columns
types_valeurs = df_cap_prod.dtypes
df_colonnes['Type_valeurs'] = types_valeurs

#   Affichage des valeurs uniques

df_colonnes.insert(loc=len(df_colonnes.columns), column='Valeurs_uniques', value='Valeurs continues')

df_colonnes.loc[(df_colonnes['Nom_colonne'] =='code_region'),'Valeurs_uniques'] = str(df_cap_prod['code_region'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='region'),'Valeurs_uniques'] = str(df_cap_prod['region'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='code_filiere'),'Valeurs_uniques'] = str(df_cap_prod['code_filiere'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='filiere_production'),'Valeurs_uniques'] = str(df_cap_prod['filiere'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='code_combustible'),'Valeurs_uniques'] = str(df_cap_prod['code_combustible'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='combustible'),'Valeurs_uniques'] = str(df_cap_prod['combustible'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='combustibles_secondaires'),'Valeurs_uniques'] = str(df_cap_prod['combustibles_secondaires'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='code_technologie'),'Valeurs_uniques'] = str(df_cap_prod['code_technologie'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='technologie'),'Valeurs_uniques'] = str(df_cap_prod['technologie'].unique())
df_colonnes.loc[(df_colonnes['Nom_colonne'] =='type_stockage'),'Valeurs_uniques'] = str(df_cap_prod['type_stockage'].unique())

pd.set_option('display.max_colwidth', None)
#   Affichage du nombre de valeurs des colonnes de df
df_colonnes['Nb_valeurs'] = df_cap_prod.count()

#   Affichage des NaN

#        Affichage du nombre de NaN de chaque colonne
df_colonnes['nb_NaN'] = df_cap_prod.isna().sum()

#        Quantité totale de 'valeurs' + 'NaN'
df_colonnes['Nb_datas'] = df_colonnes['Nb_valeurs']+df_colonnes['nb_NaN']

#       Affichage du % de données manquantes
df_colonnes['%_Valeurs'] = round(df_colonnes['Nb_valeurs']/df_colonnes['Nb_datas']*100,2)
df_colonnes['%_NaN'] =round(df_cap_prod.isna().sum() /df_colonnes['Nb_datas']*100,2)


#   Export du compte-rendu des colonnes dans un fichier Excel

file_export2 ='2_Rapp_explo_col_registre_prod.xlsx'

df_colonnes.to_excel(file_export2)

print("Rapport préliminaire d'exploration des colonnes du dataframe 'registre_national_installation_production_stockage_electricite' exporté avec succès dans le fichier Excel '2_Rapp_explo_col_registre_prod.xlsx'")

Rapport préliminaire d'exploration des colonnes du dataframe 'registre_national_installation_production_stockage_electricite' exporté avec succès dans le fichier Excel '2_Rapp_explo_col_registre_prod.xlsx'


In [9]:
# Suppression des lignes associées à des régions non traitées dans le projet Eco2Mix
regions_to_remove = ['Corse', 'Guadeloupe', 'Martinique', 'Guyane', 'La Réunion']
df_cap_prod = df_cap_prod[df_cap_prod['region'].isin(regions_to_remove) ==False]


In [10]:
print(df_cap_prod.columns.tolist())

['nomInstallation', 'code_EIC', 'codeIRIS', 'codeINSEECommune', 'commune', 'codeEPCI', 'EPCI', 'codeDepartement', 'departement', 'code_region', 'region', 'codeIRISCommuneImplantation', 'codeINSEECommuneImplantation', 'codeS3RENR', 'date_raccordement', 'date_deraccordement', 'date_mise_en_service', 'dateDebutVersion', 'posteSource', 'tensionRaccordement', 'modeRaccordement', 'code_filiere', 'filiere', 'code_combustible', 'combustible', 'codes_combustibles_secondaires', 'combustibles_secondaires', 'code_technologie', 'technologie', 'type_stockage', 'puis_max_installee', 'puis_max_rac_charge', 'puis_max_charge', 'puis_max_rac', 'puis_max_installee_discharge', 'nbGroupes', 'nbInstallations', 'regime', 'energie_stockable', 'capaciteReservoir', 'hauteurChute', 'productible', 'debitMaximal', 'codeGestionnaire', 'gestionnaire', 'energie_annuelle_glissante_injectee', 'energie_annuelle_glissante_produite', 'energie_annuelle_glissante_soutiree', 'energie_annuelle_glissante_stockee']


In [11]:
# TRAITEMENT DES DONNEES DU REGISTRE df_cap_prod

In [12]:
#   Suppression des colonnes non utiles au projet, notamment celles précisant les localisations; la présence de doublons ayant été exclue
colonnes_to_drop = [0,2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 17, 18, 19, 20, 21,23,24,25,26,27,28,29,31,33,35, 36, 37, 39, 40, 42, 43, 44,45,46,47,48]
df_cap_prod.drop(columns = df_cap_prod.columns[colonnes_to_drop], inplace =True)

df_cap_prod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71567 entries, 712 to 73692
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   code_EIC                      50282 non-null  object 
 1   region                        71536 non-null  object 
 2   date_raccordement             71305 non-null  object 
 3   date_deraccordement           40 non-null     object 
 4   date_mise_en_service          71189 non-null  object 
 5   filiere                       71567 non-null  object 
 6   puis_max_installee            71372 non-null  float64
 7   puis_max_charge               3791 non-null   float64
 8   puis_max_installee_discharge  3205 non-null   float64
 9   energie_stockable             66846 non-null  float64
 10  productible                   66837 non-null  float64
dtypes: float64(5), object(6)
memory usage: 6.6+ MB


In [13]:
df_cap_prod['filiere'].unique()

array(['Solaire', 'Hydraulique', 'Eolien', 'Thermique non renouvelable',
       'Stockage non hydraulique', 'Bioénergies', 'Nucléaire',
       'thermique', 'stockage_batterie', 'Energies Marines', 'Géothermie',
       'eolien'], dtype=object)

In [14]:
df_cap_prod['filiere'].unique()

array(['Solaire', 'Hydraulique', 'Eolien', 'Thermique non renouvelable',
       'Stockage non hydraulique', 'Bioénergies', 'Nucléaire',
       'thermique', 'stockage_batterie', 'Energies Marines', 'Géothermie',
       'eolien'], dtype=object)

In [15]:
df_cap_prod['date_raccordement'].unique()

array(['25/08/2008', '30/11/2006', '27/12/2022', ..., '27/01/2004',
       '12/08/2005', '18/01/1900'], dtype=object)

In [16]:
# GESTION DES DATES

In [17]:
# Remplacement des valeurs spécifiques dans les colonnes de date
liste_dates_replace = {'1893-01-01':'01/01/1893', '1897-03-21':'21/03/1897','1898-01-01':'01/01/1898'}
df_cap_prod.replace(liste_dates_replace, inplace=True)

In [18]:
# Conversion des colonnes de dates en format datetime
colonnes_a_convertir = ['date_raccordement', 'date_deraccordement', 'date_mise_en_service']

In [19]:
# Conversion de toutes les dates avec gestion des formats et erreurs
for colonne in colonnes_a_convertir:
    df_cap_prod[colonne] = pd.to_datetime(df_cap_prod[colonne], dayfirst=True, errors='coerce')


In [20]:
# Gestion des valeurs manquantes des colonnes de dates
df_cap_prod.loc[df_cap_prod['date_raccordement'].isna(), 'date_raccordement'] = pd.to_datetime('1800-01-01')
df_cap_prod.loc[df_cap_prod['date_deraccordement'].isna(), 'date_deraccordement'] = pd.to_datetime('2100-01-01')

In [21]:
# Remplacement des valeurs manquantes dans 'date_mise_en_service' par celles de 'date_raccordement'
df_cap_prod['date_mise_en_service'] = df_cap_prod['date_mise_en_service'].fillna(df_cap_prod['date_raccordement'])


In [22]:
# Suppression des lignes où 'date_deraccordement' est inférieure ou égale à '01/01/2013'
date_seuil = pd.to_datetime('01/01/2013', dayfirst=True)
df_cap_prod = df_cap_prod[df_cap_prod['date_deraccordement'] > date_seuil]

In [23]:
# Suppression des lignes où 'date_raccordement' est supérieure ou égale à '31/12/2022'
date_seuil2 = pd.to_datetime('31/12/2022', dayfirst=True)
df_cap_prod = df_cap_prod[df_cap_prod['date_raccordement'] < date_seuil2]


In [24]:
# Suppression des lignes où 'date_raccordement' est supérieure ou égale à '31/12/2022'
date_seuil2 = pd.to_datetime('31/12/2022', dayfirst=True)
df_cap_prod = df_cap_prod[df_cap_prod['date_raccordement'] < date_seuil2]


In [25]:
# Vérification des colonnes après traitement
print(df_cap_prod.dtypes)

code_EIC                                object
region                                  object
date_raccordement               datetime64[ns]
date_deraccordement             datetime64[ns]
date_mise_en_service            datetime64[ns]
filiere                                 object
puis_max_installee                     float64
puis_max_charge                        float64
puis_max_installee_discharge           float64
energie_stockable                      float64
productible                            float64
dtype: object


In [26]:
df_cap_prod['filiere'].unique()

array(['Solaire', 'Hydraulique', 'Eolien', 'Thermique non renouvelable',
       'Stockage non hydraulique', 'Bioénergies', 'Nucléaire',
       'thermique', 'stockage_batterie', 'Energies Marines', 'Géothermie',
       'eolien'], dtype=object)

In [27]:
#  GESTION DES COLONNES RENSEIGNANT LES COMBUSTIBLES

In [28]:
#           Remplacement des NaN des colonnes numériques par 0
columns_to_replace2 = ['puis_max_installee','puis_max_charge','puis_max_installee_discharge',
'energie_stockable','productible']
df_cap_prod[columns_to_replace2] = df_cap_prod[columns_to_replace2].fillna(0)

df_cap_prod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71532 entries, 712 to 73692
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   code_EIC                      50247 non-null  object        
 1   region                        71501 non-null  object        
 2   date_raccordement             71532 non-null  datetime64[ns]
 3   date_deraccordement           71532 non-null  datetime64[ns]
 4   date_mise_en_service          71532 non-null  datetime64[ns]
 5   filiere                       71532 non-null  object        
 6   puis_max_installee            71532 non-null  float64       
 7   puis_max_charge               71532 non-null  float64       
 8   puis_max_installee_discharge  71532 non-null  float64       
 9   energie_stockable             71532 non-null  float64       
 10  productible                   71532 non-null  float64       
dtypes: datetime64[ns](3), float64(5

In [29]:
df_cap_prod.isna().sum()

code_EIC                        21285
region                             31
date_raccordement                   0
date_deraccordement                 0
date_mise_en_service                0
filiere                             0
puis_max_installee                  0
puis_max_charge                     0
puis_max_installee_discharge        0
energie_stockable                   0
productible                         0
dtype: int64

In [30]:
df_cap_prod['filiere'].unique()

array(['Solaire', 'Hydraulique', 'Eolien', 'Thermique non renouvelable',
       'Stockage non hydraulique', 'Bioénergies', 'Nucléaire',
       'thermique', 'stockage_batterie', 'Energies Marines', 'Géothermie',
       'eolien'], dtype=object)

In [31]:
# GESTION DES REGIONS
    # Renommage des noms des régions de la colonne 'region' tels qu'ils le sont dans dataframe Eco2MixRégions traité
dictionnaire_region= {"Provence-Alpes-Côte d'Azur":"PROVENCE ALPES COTE D AZUR",
                      "Bourgogne-Franche-Comté":"BOURGOGNE FRANCHE COMTE",
                      "Bretagne":"BRETAGNE",
                      "Normandie":"NORMANDIE",
                      "Nouvelle-Aquitaine":"NOUVELLE AQUITAINE",
                      "Hauts-de-France":"HAUTS DE FRANCE",
                      "Île-de-France":"ILE DE FRANCE",
                      "Auvergne-Rhône-Alpes":"AUVERGNE RHONE ALPES",
                      "Grand Est":"GRAND EST",
                      "Centre-Val de Loire":"CENTRE VAL DE LOIRE",
                      "Occitanie":"OCCITANIE",
                      "Pays de la Loire":"PAYS DE LA LOIRE"}
df_cap_prod['region']=df_cap_prod['region'].replace(dictionnaire_region)

In [32]:
# GESTION DES NOM DES FILIERES DE PRODUCTION 

In [33]:
print("Valeurs uniques de la colonne 'filiere':", df_cap_prod['filiere'].unique())

print("Nombre de valeurs manquantes de la colonne 'filiere' :",df_cap_prod['filiere'].isna().sum())
df_cap_prod.dropna(subset=['filiere'], inplace=True)
print("Nombre de valeurs manquantes de la colonne 'filiere' :",df_cap_prod['filiere'].isna().sum())

Valeurs uniques de la colonne 'filiere': ['Solaire' 'Hydraulique' 'Eolien' 'Thermique non renouvelable'
 'Stockage non hydraulique' 'Bioénergies' 'Nucléaire' 'thermique'
 'stockage_batterie' 'Energies Marines' 'Géothermie' 'eolien']
Nombre de valeurs manquantes de la colonne 'filiere' : 0
Nombre de valeurs manquantes de la colonne 'filiere' : 0


In [34]:
 # Renommage des valeurs de 'filiere' pour équivalence avec les données de 'eco2mix-regional-cons-def'
dictionnaire_region_abr= {'Solaire' :'solaire',
                          'Hydraulique': 'hydraulique',
                          'Eolien' : 'eolien',
                          'Thermique non renouvelable':'thermique',
                          'Stockage non hydraulique' : 'stockage_batterie',
                          'Bioénergies' : 'bioenergies',
                          'Nucléaire' : 'nucleaire',
                          'Energies Marines' : 'hydraulique',
                          'Géothermie':'geothermie'}
df_cap_prod['filiere']=df_cap_prod['filiere'].replace(dictionnaire_region_abr)
df_cap_prod['filiere'].unique()


array(['solaire', 'hydraulique', 'eolien', 'thermique',
       'stockage_batterie', 'bioenergies', 'nucleaire', 'geothermie'],
      dtype=object)

In [35]:
df_cap_prod['date_raccordement'] = pd.to_datetime(df_cap_prod['date_raccordement'])
df_cap_prod['date_deraccordement'] = pd.to_datetime(df_cap_prod['date_deraccordement'])
df_cap_prod['date_mise_en_service'] = pd.to_datetime(df_cap_prod['date_mise_en_service'])

df_cap_prod.info()


<class 'pandas.core.frame.DataFrame'>
Index: 71532 entries, 712 to 73692
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   code_EIC                      50247 non-null  object        
 1   region                        71501 non-null  object        
 2   date_raccordement             71532 non-null  datetime64[ns]
 3   date_deraccordement           71532 non-null  datetime64[ns]
 4   date_mise_en_service          71532 non-null  datetime64[ns]
 5   filiere                       71532 non-null  object        
 6   puis_max_installee            71532 non-null  float64       
 7   puis_max_charge               71532 non-null  float64       
 8   puis_max_installee_discharge  71532 non-null  float64       
 9   energie_stockable             71532 non-null  float64       
 10  productible                   71532 non-null  float64       
dtypes: datetime64[ns](3), float64(5

In [36]:
df_cap_prod['date_deraccordement'].isna().sum()

0

In [37]:
# ================================================= CALCUL DES CAPACITES INSTALLEES PAR JOUR, REGION ET FILIERE 

In [ ]:
# Création de la plage de dates
dates = pd.date_range(start='2013-01-01', end='2023-12-31', freq='D').strftime('%Y-%m-%d')
regions = df_cap_prod['region'].unique()
filieres = ['thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies']

def get_cap_prod_max(region, date, filiere):
    mask = (
        (df_cap_prod['region'] == region) &
        (df_cap_prod['filiere'] == filiere) &
        (df_cap_prod['date_raccordement'] <= date) &
        (df_cap_prod['date_deraccordement'] > date)
    )
    return df_cap_prod.loc[mask, 'puis_max_installee'].sum() / 1000

df_result_cap_prod = []

total_iterations = len(regions) * len(dates)
progress = tqdm(total=total_iterations, desc="Calcul en cours")

for region in regions:
    for date in dates:
        row = {'date': date, 'region': region}
        for filiere in filieres:
            row[f'cap_prod_max_{filiere}'] = get_cap_prod_max(region, date, filiere)
        df_result_cap_prod.append(row)
        progress.update(1)  # Mise à jour

progress.close()

df_result_cap_prod = pd.DataFrame(df_result_cap_prod)
df_result_cap_prod.to_csv('df_result_cap_prod_2013_2023.csv', index=False)

print("Exécution terminée avec succès.")


Calcul en cours:   4%|▎         | 1860/52221 [01:54<52:34, 15.97it/s]  

In [ ]:
# Export dans un classer Excel

# Liste des filières à traiter
filieres = ['thermique', 'nucleaire', 'eolien', 'solaire', 'hydraulique', 'bioenergies']

# Création du fichier Excel avec une feuille par filière
with pd.ExcelWriter('df_result_cap_prod_2013_2023.xlsx', engine='openpyxl') as writer:
    for filiere in filieres:
        # Création d'un sous-dataframe pour chaque filière
        df_filiere = df_result_cap_prod.pivot(index='date', columns='region', values=f'cap_prod_max_{filiere}')
        
        # Ajout de la feuille dans le fichier Excel
        df_filiere.to_excel(writer, sheet_name=filiere)

print("Fichier Excel généré avec succès.")

In [ ]:
df_result_cap_prod.info()

In [ ]:
# Export des 100 premières lignes dans un nouveau fichier CSV pour aperçu
df_result_cap_prod_head = df_result_cap_prod.head(100)
df_result_cap_prod_head.to_csv('df_result_cap_prod_first_100_lines.csv', sep=';', encoding='latin-1', index=False)